In [ ]:
!pip install tensorflow

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
json_path = os.path.join("/content", "data.json")

In [ ]:
with open(json_path, 'r') as file:
    data = json.load(file)

In [ ]:
patterns = []
tags = []

for intent in data["intens"]:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tags"])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
X = tokenizer.texts_to_sequences(patterns)

In [ ]:
X_padded = pad_sequences(X)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(tags)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=X_padded.shape[1]))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(len(set(y_encoded)), activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_padded, y_encoded, epochs=25, batch_size=1, verbose=2)

Epoch 1/25
46/46 - 1s - loss: 2.7758 - accuracy: 0.0435 - 808ms/epoch - 18ms/step
Epoch 2/25
46/46 - 0s - loss: 2.7304 - accuracy: 0.2609 - 63ms/epoch - 1ms/step
Epoch 3/25
46/46 - 0s - loss: 2.6796 - accuracy: 0.2391 - 65ms/epoch - 1ms/step
Epoch 4/25
46/46 - 0s - loss: 2.6017 - accuracy: 0.3478 - 65ms/epoch - 1ms/step
Epoch 5/25
46/46 - 0s - loss: 2.4964 - accuracy: 0.4783 - 78ms/epoch - 2ms/step
Epoch 6/25
46/46 - 0s - loss: 2.3482 - accuracy: 0.4565 - 68ms/epoch - 1ms/step
Epoch 7/25
46/46 - 0s - loss: 2.1745 - accuracy: 0.4783 - 68ms/epoch - 1ms/step
Epoch 8/25
46/46 - 0s - loss: 1.9523 - accuracy: 0.5652 - 70ms/epoch - 2ms/step
Epoch 9/25
46/46 - 0s - loss: 1.7189 - accuracy: 0.5870 - 67ms/epoch - 1ms/step
Epoch 10/25
46/46 - 0s - loss: 1.4832 - accuracy: 0.7826 - 65ms/epoch - 1ms/step
Epoch 11/25
46/46 - 0s - loss: 1.2579 - accuracy: 0.7826 - 64ms/epoch - 1ms/step
Epoch 12/25
46/46 - 0s - loss: 1.0471 - accuracy: 0.8261 - 69ms/epoch - 2ms/step
Epoch 13/25
46/46 - 0s - loss: 0.86

In [ ]:
def get_response(user_input):
    user_input_sequence = tokenizer.texts_to_sequences([user_input])
    user_input_padded = pad_sequences(user_input_sequence, maxlen=X_padded.shape[1])
    predicted_probabilities = model.predict(user_input_padded)
    predicted_class = np.argmax(predicted_probabilities, axis=-1)
    predicted_tag = label_encoder.inverse_transform(predicted_class)

    for intent in data["intens"]:
        if intent["tags"] == predicted_tag:
            return intent["responses"]

In [ ]:
user_input = "berapa lama pembuatan ktp"
response = get_response(user_input)
print(response)

1/1 [==============================] - 0s 146ms/step
Durasi proses pembuatan E-KTP tergantung pada lokasi dan tingkat kepadatan penduduk, Bagi warga negara yang sudah memenuhi syarat, maka pembuatan KTP dilakukan selambat-lambatnya 14 hari
